In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
%matplotlib inline

# Web scraping

In [ ]:
chromedriver = "/Users/jaykim/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
# variables that will save data from the web scraping
yeartrim=[]
price=[]
miles=[]
exterior=[]
interior=[]
transmission=[]
drivetrain=[]

In [ ]:
driver = webdriver.Chrome(chromedriver)
time.sleep(1);

# add1 and add2 will be used as links to go through pages on cars.com
add1='https://www.cars.com/for-sale/searchresults.action/?mdId=20823&mkId=20017&page='  
add2='&perPage=100&rd=99999&searchSource=PAGINATION&sort=relevance&stkTypId=28881&zc=98109'   

reference=['Exterior','Interior','Transmission','Drivetrain']*100   # There are missing information on web page. This variable will clean them up.
for k in range(21):    # Scraping 21 pages
    url=add1+str(k+1)+add2
    driver.get(url)
    time.sleep(1);

    a=driver.find_elements_by_xpath('//h2[@class="cui-delta listing-row__title"]')   # getting info from the webpage
    b=driver.find_elements_by_xpath('//span[@class="listing-row__price"]')           # getting info from the webpage
    c=driver.find_elements_by_xpath('//span[@class="listing-row__mileage"]')         # getting info from the webpage
    d=driver.find_elements_by_xpath('//span[@class="listing-row__meta-item"]')       # getting info from the webpage
     
    for i in range(len(a)):          
        yeartrim.append(a[i].text)
        price.append(b[i].text)
        miles.append(c[i].text)

    for i in range(len(reference)):   # To account for missing values
        if reference[i] not in d[i].text:
            d.insert(i, d[i-1])
        
    for i in range(len(d)//4):
        exterior.append(d[4*i].text)
        interior.append(d[4*i+1].text)
        transmission.append(d[4*i+2].text)
        drivetrain.append(d[4*i+3].text)
    print(len(d),',',len(exterior),',',len(interior),',',len(transmission),',',len(drivetrain))   

In [ ]:
label=['yeartrim','price','miles','exterior','interior','transmission','drivetrain']   # Create DataFrame
matrix = np.matrix([yeartrim, price, miles, exterior, interior, transmission, drivetrain])
df = pd.DataFrame(data=matrix.T, columns=label)

In [ ]:
# Look for missing information and remove rows if there are

df['bad'] = df.interior.str.split(': ').str.get(0)        
df3 = df.loc[df.bad == 'Interior Color']
len(df3)
df3['bad2'] = df3.exterior.str.split(': ').str.get(0)
df4 = df3.loc[df3.bad2 == 'Exterior Color']
len(df4)
df5 = df4.drop(['bad','bad2','bad3'],axis=1)

In [ ]:
yeartrim = df5.yeartrim.tolist()
pr=df5.price.tolist()
mi=df5.miles.tolist()
exte=df5.exterior.tolist()
inte=df5.interior.tolist()
trans=df5.transmission.tolist()
drive=df5.drivetrain.tolist()

In [ ]:
# Clean up data

year = [int(yeartrim[i].split(' ')[1]) for i in range(len(yeartrim))]
trim = [yeartrim[i].split(' ')[-1] for i in range(len(yeartrim))]
for i in range(len(pr)):
    if pr[i] == 'Not Priced':
        pr[i] = pr[i].replace('Not Priced','$0,')
price = [int(pr[i].replace('$','').replace(',','')) for i in range(len(pr))]
miles = [int(mi[i].split(': ')[-1].replace(',','')) for i in range(len(mi))]
exterior = [exte[i].split(': ')[-1] for i in range(len(exte))]
interior = [inte[i].split(': ')[-1] for i in range(len(inte))]
transmission = [trans[i].split(': ')[-1] for i in range(len(trans))]

In [ ]:
# Put all data into DataFrame

label=['year','trim','price','miles','exterior','interior','transmission']
data1 =[year,trim,price,miles,exterior,interior,transmission]

df7=pd.DataFrame(data=data1, columns=label)

In [ ]:
df=pd.DataFrame.copy(df7)

In [ ]:
# Remove one outlier
df2=df.loc[df.price<30000]
df2.head()

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(3,1,1)
plt.hist(df2.year, bins=17)
plt.title('Year', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.subplot(3,1,2)
plt.hist(df2.miles, bins=17)
plt.title('Miles', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.subplot(3,1,3)
plt.hist(df2.price, bins=17)
plt.title('Price', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(3,1,1)
plt.scatter(df2.year, df2.miles)
plt.title('year  miles')
plt.subplot(3,1,2)
plt.scatter(df2.miles, df2.price)
plt.title('miles  price')
plt.subplot(3,1,3)
plt.scatter(df2.year, df2.price)
plt.title('year  price')

# Data Clean up

In [ ]:
# Categorical variables have too many features and typos. This will clean up data and simplyfy the categories.
# For transmission

df2.transmission.replace('1-Speed CVT w/OD','Automatic CVT',inplace=True)
df2.transmission.replace('CVT w/OD','Automatic CVT',inplace=True)
df2.transmission.replace('continuously variable automatic','Automatic CVT',inplace=True)
df2.transmission.replace('5-Speed Automatic','Automatic',inplace=True)
df2.transmission.replace('6-Speed Manual','Manual',inplace=True)
df2.transmission.replace('5-Speed Manual','Manual',inplace=True)
df2.transmission.replace('4-Speed Automatic','Automatic',inplace=True)
df2.transmission.replace('Variable','Automatic CVT',inplace=True)
df2.transmission.replace('6-Speed Manual w/OD','Manual',inplace=True)
df2.transmission.replace('5 speed automatic','Automatic',inplace=True)
df2.transmission.replace('1-Speed Variable','Automatic CVT',inplace=True)
df2.transmission.replace('5-Speed Manual w/OD','Manual',inplace=True)
df2.transmission.replace('4 speed automatic','Automatic',inplace=True)
df2.transmission.replace('4-Speed Electronic Automatic','Automatic',inplace=True)
df2.transmission.replace('Auto CD, REAR CAM, CLEAN CAR, BLUETOOTH','Automatic',inplace=True)
df2.transmission.replace('6 speed manual','Manual',inplace=True)
df2.transmission.replace('6-Speed Automatic','Automatic',inplace=True)
df2.transmission.replace('Close-Ratio 6-Speed Manual','Manual',inplace=True)
df2.transmission.replace('Automatic CVT','CVT',inplace=True)

In [ ]:
# For interior color

df2.interior.replace('GRAY','Gray',inplace=True)
df2.interior.replace('gray','Gray',inplace=True)
df2.interior.replace('Stone Gray','Gray',inplace=True)
df2.interior.replace('Stone','Gray',inplace=True)
df2.interior.replace('Light Gray','Gray',inplace=True)
df2.interior.replace('Slate Gray','Gray',inplace=True)
df2.interior.replace('Black / Gray','Gray / Black',inplace=True)
df2.interior.replace('Black/Gray','Gray / Black',inplace=True)
df2.interior.replace('Black / Red','Black/Red',inplace=True)

In [ ]:
# For trim

df2.trim.replace('EX-L','EX',inplace=True)
df2.trim.replace('EX-T','EX',inplace=True)
df2.trim.replace('LX-S','LX',inplace=True)
df2.trim.replace('LX-P','LX',inplace=True)
df2.trim.replace('DX-VP','DX',inplace=True)

In [ ]:
# For exterior color

df2.exterior.replace('Taffeta White','White',inplace=True)
df2.exterior.replace('White Orchid','White',inplace=True)
df2.exterior.replace('Galaxy Gray Metallic','Gray',inplace=True)
df2.exterior.replace('Sonic Gray Pearl','Gray',inplace=True)
df2.exterior.replace('Dark Gray','Gray',inplace=True)
df2.exterior.replace('Ghost Gray','Gray',inplace=True)
df2.exterior.replace('Atomic Blue Metallic','Blue',inplace=True)
df2.exterior.replace('Aegean Blue Metallic','Blue',inplace=True)
df2.exterior.replace('Dyno Blue Pearl','Blue',inplace=True)
df2.exterior.replace('Royal Blue Pearl','Blue',inplace=True)
df2.exterior.replace('Dyno Blue Pearl Ii','Blue',inplace=True)
df2.exterior.replace('Aegean Blue','Blue',inplace=True)
df2.exterior.replace('Dyno Blue','Blue',inplace=True)
df2.exterior.replace('Blue Pearl','Blue',inplace=True)
df2.exterior.replace('Fiji Blue Pearl','Blue',inplace=True)
df2.exterior.replace('Cosmic Blue Metallic','Blue',inplace=True)
df2.exterior.replace('Tango Red Pearl','Red',inplace=True)
df2.exterior.replace('Dark Red','Red',inplace=True)
df2.exterior.replace('Rally Red','Red',inplace=True)
df2.exterior.replace('Milano Red','Red',inplace=True)
df2.exterior.replace('Crimson Pearl Red','Red',inplace=True)
df2.exterior.replace('Rallye Red','Red',inplace=True)

In [ ]:
df2.exterior.replace('Alabaster Silver Metallic','Silver',inplace=True)
df2.exterior.replace('Lunar Silver Metallic','Silver',inplace=True)
df2.exterior.replace('Satin Silver Metallic','Silver',inplace=True)
df2.exterior.replace('Silver Mist','Silver',inplace=True)
df2.exterior.replace('Alabaster Silver','Silver',inplace=True)
df2.exterior.replace('Silver Metallic','Silver',inplace=True)
df2.exterior.replace('Lunar Silver','Silver',inplace=True)
df2.exterior.replace('White Orchid Pearl','White',inplace=True)
df2.exterior.replace('Crystal Black Pearl','Black',inplace=True)
df2.exterior.replace('Nighthawk Black Pearl','Black',inplace=True)
df2.exterior.replace('Crystal Black','Black',inplace=True)
df2.exterior.replace('Super Black','Black',inplace=True)
df2.exterior.replace('Nighthawk Black','Black',inplace=True)

In [ ]:
df2.exterior.replace('Modern Steel Metallic','Metallic',inplace=True)
df2.exterior.replace('Urban Titanium Metallic','Metallic',inplace=True)
df2.exterior.replace('Polished Metal Metallic','Metallic',inplace=True)
df2.exterior.replace('Polished Metal','Metallic',inplace=True)
df2.exterior.replace('Modern Steel','Metallic',inplace=True)
df2.exterior.replace('Urban Titanium','Metallic',inplace=True)
df2.exterior.replace('Kona Coffee Metallic','Metallic',inplace=True)
df2.exterior.replace('Magnesium Metallic','Metallic',inplace=True)
df2.exterior.replace('Borrego Beige Metallic','Metallic',inplace=True)
df2.exterior.replace('Shoreline Mist Metallic','Metallic',inplace=True)
df2.exterior.replace('Titanium Metallic','Metallic',inplace=True)
df2.exterior.replace('Cool Mist Metallic','Metallic',inplace=True)

In [ ]:
# Add a category 'Other' to merge the minor categories

index_exterior = df2.exterior.value_counts()[:8]
for i in range(len(df2)):
    if df2.exterior[i] not in index_exterior:
        df2.exterior.replace(df2.exterior[i],'Other',inplace=True)

index_interior = df2.interior.value_counts()[:9]
for i in range(len(df2)):
    if df2.interior[i] not in index_interior:
        df2.interior.replace(df2.interior[i],'Other',inplace=True)

# Dummy variables for categorical data and adding more complexity for numerical data

In [ ]:
X=df2[['year','miles']]

# Generate dummy variables for categorical data

dummy_trim=pd.get_dummies(df2.trim, prefix='trim')#, drop_first=True)
dummy_exterior=pd.get_dummies(df2.exterior, prefix='exterior')#, drop_first=True)
dummy_interior=pd.get_dummies(df2.interior, prefix='interior')#, drop_first=True)
dummy_transmission=pd.get_dummies(df2.transmission, prefix='transmission')#, drop_first=True)

X = X.join(dummy_trim)
X = X.join(dummy_exterior)
X = X.join(dummy_interior)
X = X.join(dummy_transmission)

y=df2.price.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=42)

# Log transformation for year and miles to remove the bias from data

X_train['year_log'] = np.log(X_train.year)
X_train['miles_log'] = np.log(X_train.miles)
X_train = X_train.drop(['year','miles'],axis=1)

X_test['year_log'] = np.log(X_test.year)
X_test['miles_log'] = np.log(X_test.miles)
X_test = X_test.drop(['year','miles'],axis=1)

In [ ]:
# Adding more complexity for year and miles to account for nonlinear relationship

X2_train = X_train.filter(['year_log','miles_log'], axis=1)
X2_test = X_test.filter(['year_log','miles_log'], axis=1)

p = PolynomialFeatures(2)
X2_train = p.fit_transform(X2_train)
X2_test = p.transform(X2_test)

# Normalization
s = StandardScaler()
X2_train_co = s.fit_transform(X2_train)
X2_test_co = s.transform(X2_test)

# Formatting data
col = ['col'+str(i) for i in range(6)]
X2_train_co = pd.DataFrame(columns=col, data=X2_train_co)
X2_test_co = pd.DataFrame(columns=col, data=X2_test_co)

X2_train_complex = pd.DataFrame.copy(X_train)
X2_test_complex = pd.DataFrame.copy(X_test)

X2_train_complex = X2_train_complex.reset_index(drop=True)
X2_test_complex = X2_test_complex.reset_index(drop=True)

X2_train_complex = X2_train_complex.join(X2_train_co)
X2_test_complex = X2_test_complex.join(X2_test_co)

X2_train_complex = X2_train_complex.drop(['year_log','miles_log'],axis=1)
X2_test_complex = X2_test_complex.drop(['year_log','miles_log'],axis=1)
X2_train_complex.col0.replace(0,1,inplace=True)

X2_train_complex.rename(columns={'col0': 'intercept', 'col1': 'year','col2':'mile','col3':'year_2','col4':'yearmile','col5':'mile_2'}, inplace=True)

In [ ]:
# Regularization with Ridge regression

est = make_pipeline(RidgeCV(cv=3,alphas=(1e-8,1e-4,1e0,1e4,1e8)))
est.fit(X2_train_complex, y_train)

print('Train R^2: ',est.score(X2_train_complex, y_train))
print('Train RMSSE:', np.sqrt(mean_squared_error(y_train, est.predict(X2_train_complex))))
print('Test R^2: ', est.score(X2_test_complex, y_test))
print('Test RMSSE:', np.sqrt(mean_squared_error(y_test, est.predict(X2_test_complex))))

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(y_test, est.predict(X2_test_complex), label='Data')
a=[0,25000]
b=[0,25000]
plt.plot(a,b, color='red', label='Perfect match')
plt.title('Price Prediction', fontsize=30)
plt.xlabel('Actual Price ($)',fontsize=20)
plt.ylabel('Predicted Price ($)',fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(est.predict(X2_train_complex), est.predict(X2_train_complex) - y_train, c='b',alpha = 0.5, label='Train')
plt.scatter(est.predict(X2_test_complex), est.predict(X2_test_complex) - y_test, c='g',alpha = 0.5, label='Test')
plt.title('Residual scatter plot for Train (blue) and Test (green) data', fontsize=30)
plt.xlabel('Predicted Price ($)',fontsize=20)
plt.ylabel('Residual ($)',fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim([-5000,25000])

# Calculate p-values with Statsmodel

In [ ]:
X3_train_complex = X2_train_complex.reset_index(drop=True)
stats_y = pd.DataFrame(y_train, columns=['price'])
X3_train_complex = X3_train_complex.join(stats_y)

# Generate equations for Statsmodel
equ='price ~ ' + str(X3_train_complex.columns[0])
for i in range(1,X3_train_complex.shape[1]-1):
    equ += ' + '
    equ += str(X3_train_complex.columns[i])
    
y, X = patsy.dmatrices(equ, data=X3_train_complex, return_type="dataframe")
model = sm.OLS(y, X)
fit = model.fit()

fit.summary()

# Study with different degrees of polynomial features

In [ ]:
r2_train=[]
r2_test=[]
rmse_train=[]
rmse_test=[]
index=[]
for k in range(2,10):  # Exploring different degrees from 2 to 9
    p = PolynomialFeatures(k)
    X3_train = p.fit_transform(X2_train)
    X3_test = p.transform(X2_test)
    s = StandardScaler()
    X1_train_co = s.fit_transform(X3_train)
    X1_test_co = s.transform(X3_test)

    X2_train_co = pd.DataFrame( data=X1_train_co) #columns=col
    X2_test_co = pd.DataFrame(data=X1_test_co) #columns=col

    X2_train_complex = pd.DataFrame.copy(X_train)
    X2_test_complex = pd.DataFrame.copy(X_test)

    X2_train_complex = X2_train_complex.reset_index(drop=True)
    X2_test_complex = X2_test_complex.reset_index(drop=True)

    X2_train_complex = X2_train_complex.join(X2_train_co)
    X2_test_complex = X2_test_complex.join(X2_test_co)
    X2_train_complex = X2_train_complex.drop(['year_log','miles_log'],axis=1)
    X2_test_complex = X2_test_complex.drop(['year_log','miles_log'],axis=1)

    est = make_pipeline(RidgeCV(cv=3,alphas=(1e-8,1e-4,1e0,1e4,1e8)))

    est.fit(X2_train_complex, y_train)
    print('complexity =', k)
    print('Train R^2: ',est.score(X2_train_complex, y_train))
    print('Train RMSSE:', np.sqrt(mean_squared_error(y_train, est.predict(X2_train_complex))))
    print('Test R^2: ', est.score(X2_test_complex, y_test))
    print('Test RMSSE:', np.sqrt(mean_squared_error(y_test, est.predict(X2_test_complex))))
    print('----------')
    
    r2_train.append(est.score(X2_train_complex, y_train))
    r2_test.append(est.score(X2_test_complex, y_test))
    rmse_train.append(np.sqrt(mean_squared_error(y_train, est.predict(X2_train_complex))))
    rmse_test.append(np.sqrt(mean_squared_error(y_test, est.predict(X2_test_complex))))
    index.append(k)

# FINAL PREDICTION

In [ ]:
wife_car_final=np.array([1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,-0.5111072,  0.65581954, -0.51126046,  0.65604447,0.64537731])

In [ ]:
np.dot(fit.params,wife_car_final)